In [ ]:
#%pip install gensim

In [ ]:
#%pip install spacy

In [ ]:
#%pip install pyLDAvis 

In [1]:
#Imports
import pandas as pd
import pickle
import os
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('words')
import pandas as pd
from nltk.tokenize import PunktSentenceTokenizer, RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import PorterStemmer
import matplotlib.pyplot as plt

import gensim
from gensim.models.ldamodel import LdaModel
import gensim.corpora as corpora
from gensim.models import CoherenceModel
from gensim import matutils

import spacy

import pyLDAvis
import pyLDAvis.gensim_models

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\louis\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\louis\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\louis\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [2]:
#Import soptify + genius dataset with song lyrics
df_lyrics = pd.read_pickle("df_rap_7k_nas.pickle")
df_lyrics = df_lyrics.loc[df_lyrics.lyrics.astype(str).drop_duplicates(keep=False).index].reset_index(drop=True)


In [3]:
df_lyrics.head()

,track_id,album_id,artist_id,duration_ms,flg_explicit,url,track_name,track_popularity,track_type,track_danceability,...,artist_name,artist_genres,artist_followers,artist_popularity,lyrics,lyrics_clean,lyrics_clean_split,lyrics_clean_stemmed,lyrics_wordlen,track_release_decade
0,7u3GcvpGFbodFKigTg7s2S,0cLzuJG2UDa0axMQkF7JR6,20qISvAhX20dpIbOOzGK3q,312893,True,https://api.spotify.com/v1/tracks/7u3GcvpGFbod...,My Country (feat. Millennium Thug),33,track,0.881,...,Nas,[nan],2709016,78,"My Country Lyrics\nAmerican born, American rai...",country american born american raised american...,"[country, american, born, american, raised, am...","[countri, american, born, american, rais, amer...",405,00
1,5ZagzwI4jeOHcy0BPQVOvy,7sHrPadM39466NY7DW3s4r,20qISvAhX20dpIbOOzGK3q,243240,True,https://api.spotify.com/v1/tracks/5ZagzwI4jeOH...,Blunt Ashes,32,track,0.777,...,Nas,[nan],2709016,78,"Blunt Ashes Lyrics\nYo, I wonder if Langston H...",blunt ashes wonder hughes haley got told get t...,"[blunt, ashes, wonder, hughes, haley, got, tol...","[blunt, ash, wonder, hugh, haley, got, told, g...",275,00
2,1UgmACeNYIaoDcRFBaffs8,0jghcWTsQzux5T9sAfZO13,20qISvAhX20dpIbOOzGK3q,206466,True,https://api.spotify.com/v1/tracks/1UgmACeNYIao...,Me & You (Dedicated To Destiny),23,track,0.690,...,Nas,[nan],2709016,78,Me & You (Dedicated to Destiny) Lyrics\nI'd li...,dedicated destiny like dedicate one star someo...,"[dedicated, destiny, like, dedicate, one, star...","[dedic, destini, like, dedic, one, star, someo...",146,00
3,3JiGnXAkY3d0M6Hsi6LTQ0,586sDXSThASjmWwSYnHbqP,0NbfKEOTQCcwd6o7wSDOHI,30386,False,https://api.spotify.com/v1/tracks/3JiGnXAkY3d0...,Don't Talk,13,track,0.541,...,The Game,[nan],3787921,79,What I Want Lyrics\nI think the feds is buggin...,want think phone think phone talking long thin...,"[want, think, phone, think, phone, talking, lo...","[want, think, phone, think, phone, talk, long,...",162,00
4,6w9lmWBgqzPQHNQo5KaORN,66EwBbt2kPgugo8Wz0SKAw,20qISvAhX20dpIbOOzGK3q,201560,True,https://api.spotify.com/v1/tracks/6w9lmWBgqzPQ...,Bonjour,43,track,0.647,...,Nas,[nan],2709016,78,Bonjour Lyrics\n(Bonjour) I should be more mat...,mature sure spoiled hit south france tour many...,"[mature, sure, spoiled, hit, south, france, to...","[matur, sure, spoil, hit, south, franc, tour, ...",133,10


In [4]:
#Create lemmatizer, stemmer and define stopwords

word_tokenizer=RegexpTokenizer(r'\w+')

wordnet_lemmatizer = WordNetLemmatizer()

stemmer = PorterStemmer()

stopwords_nltk = set(stopwords.words('english'))



In [5]:
#Save lyrics
temp = df_lyrics["lyrics_clean"].copy()

In [6]:
### Function to lemmatize tokenized documents (song lyrics here)
def lemmatizer(df):
    
    lemmed = []
    
    for song in df.lyrics_clean_split:
        lemmed.append([wordnet_lemmatizer.lemmatize(word) for word in song])
    return lemmed

In [7]:
### Fucntion to get dictionary and bag of words
#Returns dictionary and bow corpus
def get_dict_and_bow(data, extreme_list = None):
    
    #Create dictionary of words: mapping between word and id
    id2word = corpora.Dictionary(data)
    if extreme_list is not None:
        id2word.filter_extremes(no_below = extreme_list[0], no_above = extreme_list[1]) #Filter tokens that appear in less than and more than
    texts = data
    
    #Transform dictionary into bag of words
    corpus = [id2word.doc2bow(text) for text in texts]
    
    return id2word, corpus

In [8]:
### Function to get coherence score for particular model, dict and bow
#Returns the coherence model
def get_coherence(model, text, id_dict, method='c_v'):
    coherence_model = CoherenceModel(model=model, texts=text, dictionary=id_dict, coherence=method)
    coherence_lda = coherence_model.get_coherence()
    print(coherence_lda)
    return coherence_model

In [69]:
### Function to join document-topic scores back to document dataset
#Args: dataframe to join back into, corpus/bow, model to extract topics from 
def get_topics_in_df(df, corpus, model, num_topics):

    #set minimum probability to 0 to ensure all topic scores are extracted
    get_document_topics = [model.get_document_topics(item, minimum_probability = -1, minimum_phi_value = -1) for item in corpus] 
    document_topic_list = get_document_topics
    
    temp_df = []
    for i in range(len(get_document_topics)):
        temp_list = []
        for j in range(len(get_document_topics[i])):
            temp_tuple = get_document_topics[i][j]
            temp_list.append(temp_tuple[1])
        temp_df.append(temp_list)
    temp_df = pd.DataFrame(temp_df)
    df = df.join(temp_df)
    
    df.iloc[:,-num_topics:] = 0
    
    j = 0
    for document in document_topic_list:
        for tup in document:
            df.iloc[j, (len(df.columns) - (num_topics - int(tup[0]))) ] = float(tup[1])
    
        j += 1
        
    return df

In [11]:
lyrics_lemmatized_all = lemmatizer(df_lyrics)

In [12]:
#Create new column in df_lyrics with lemmed words
df_lyrics['lyrics_clean_split_lemmed'] = lyrics_lemmatized_all

In [13]:
id2word_all, corpus_all = get_dict_and_bow(lyrics_lemmatized_all, [200, 0.95])

In [14]:
#Fit LDA model
ntopics = 8
lda_model_all = LdaModel(corpus = corpus_all,
                    id2word = id2word_all,
                    num_topics = ntopics, 
                    random_state = 123,
                    passes=4,
                    alpha=0,
                    eta=2,
                    per_word_topics=True)

C:\Users\louis\anaconda3\envs\INSY669\lib\site-packages\gensim\models\ldamodel.py:1124: RuntimeWarning: invalid value encountered in multiply
  score += np.sum((self.alpha - gammad) * Elogthetad)
C:\Users\louis\anaconda3\envs\INSY669\lib\site-packages\gensim\models\ldamodel.py:1125: RuntimeWarning: invalid value encountered in subtract
  score += np.sum(gammaln(gammad) - gammaln(self.alpha))


In [15]:
#Show the LDA topics: topic 0
lda_model_all.show_topic(topicid=0, topn=20)


[('ooh', 0.035823893),
 ('want', 0.032349944),
 ('know', 0.03130655),
 ('right', 0.02871497),
 ('get', 0.025003666),
 ('let', 0.021404438),
 ('got', 0.020763705),
 ('night', 0.01782168),
 ('back', 0.017694026),
 ('like', 0.014951322),
 ('need', 0.014167549),
 ('take', 0.013926651),
 ('girl', 0.012973367),
 ('baby', 0.012844852),
 ('tell', 0.012300544),
 ('shit', 0.011896886),
 ('cause', 0.011040311),
 ('come', 0.010931538),
 ('love', 0.010408549),
 ('high', 0.009708414)]

In [16]:
#Get coherence score
coherence_model_all = get_coherence(lda_model_all, text = lyrics_lemmatized_all, id_dict=id2word_all)

0.3381297852996724


In [17]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis_all = pyLDAvis.gensim_models.prepare(lda_model_all, corpus_all, id2word_all)
vis_all

C:\Users\louis\anaconda3\envs\INSY669\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.060930  0.037908       1        1  21.463772
5      0.086545  0.041508       2        1  18.036525
7     -0.041968 -0.029211       3        1  16.691333
6     -0.006743  0.030523       4        1  10.546933
1      0.122899 -0.085502       5        1   9.617075
2      0.030685  0.082248       6        1   8.814888
0      0.005665 -0.034318       7        1   8.401956
4     -0.136153 -0.043157       8        1   6.427518, topic_info=      Term          Freq         Total Category  logprob  loglift
123   yeah  26429.000000  26429.000000  Default  30.0000  30.0000
698   feat   3514.000000   3514.000000  Default  29.0000  29.0000
77   nigga  13304.000000  13304.000000  Default  28.0000  28.0000
198   love   8103.000000   8103.000000  Default  27.0000  27.0000
5    bitch  11662.000000  11662.000000  Default  26.0000  26.0000
..     ...           ...           ...      ...      ...      ...
348    let    396.571311   7732.165447   Topic8  -5.0327  -0.2257
69     man    376.959641   6750.229724   Topic8  -5.0835  -0.1406
76   never    377.712086   6972.687991   Topic8  -5.0815  -0.1710
78   night    325.181532   2913.480667   Topic8  -5.2312   0.5519
312   need    320.515435   5349.145923   Topic8  -5.2457  -0.0702

[471 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
428       1  0.185286   air
428       2  0.133469   air
428       3  0.054958   air
428       4  0.406687   air
428       5  0.028264   air
...     ...       ...   ...
645       4  0.171893  zone
645       5  0.017189  zone
645       6  0.096690  zone
645       7  0.238502  zone
645       8  0.045122  zone

[2432 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 6, 8, 7, 2, 3, 1, 5])

In [25]:
#Name the topics
topic_names = {
    0:"Musicality/Flow/Lyricism",
    1:"Flex/Money",
    2:"Romance/Feelings", 
    3:"Struggle/Grind/Hustle",
    4:"Trap/Drug",
    5:"Violence/Gangster",
    6:"Party/Sexuality",
    7:"Feats/Artists/Shoutouts"
}

In [20]:
#Join topics back to dataframe
df_lyrics_topics = df_lyrics.copy()
df_lyrics_topics = get_topics_in_df(df = df_lyrics, corpus = corpus_all, model = lda_model_all, num_topics = 8)

In [26]:
df_lyrics_topics.rename(columns = topic_names, inplace = True)

In [27]:
df_lyrics_topics.head()

,track_id,album_id,artist_id,duration_ms,flg_explicit,url,track_name,track_popularity,track_type,track_danceability,...,track_release_decade,lyrics_clean_split_lemmed,Musicality/Flow/Lyricism,Flex/Money,Romance/Feelings,Struggle/Grind/Hustle,Trap/Drug,Violence/Gangster,Party/Sexuality,Feats/Artists/Shoutouts
0,7u3GcvpGFbodFKigTg7s2S,0cLzuJG2UDa0axMQkF7JR6,20qISvAhX20dpIbOOzGK3q,312893,True,https://api.spotify.com/v1/tracks/7u3GcvpGFbod...,My Country (feat. Millennium Thug),33,track,0.881,...,00,"[country, american, born, american, raised, am...",0.000000,0.0,0.000285,0.387158,0.000000,0.042575,0.000000,0.569982
1,5ZagzwI4jeOHcy0BPQVOvy,7sHrPadM39466NY7DW3s4r,20qISvAhX20dpIbOOzGK3q,243240,True,https://api.spotify.com/v1/tracks/5ZagzwI4jeOH...,Blunt Ashes,32,track,0.777,...,00,"[blunt, ash, wonder, hughes, haley, got, told,...",0.000000,0.0,0.125909,0.269506,0.000000,0.000000,0.042631,0.561954
2,1UgmACeNYIaoDcRFBaffs8,0jghcWTsQzux5T9sAfZO13,20qISvAhX20dpIbOOzGK3q,206466,True,https://api.spotify.com/v1/tracks/1UgmACeNYIao...,Me & You (Dedicated To Destiny),23,track,0.690,...,00,"[dedicated, destiny, like, dedicate, one, star...",0.000000,0.0,0.000000,0.000000,0.284339,0.000000,0.000000,0.715661
3,3JiGnXAkY3d0M6Hsi6LTQ0,586sDXSThASjmWwSYnHbqP,0NbfKEOTQCcwd6o7wSDOHI,30386,False,https://api.spotify.com/v1/tracks/3JiGnXAkY3d0...,Don't Talk,13,track,0.541,...,00,"[want, think, phone, think, phone, talking, lo...",0.057499,0.0,0.000000,0.000000,0.000000,0.318749,0.213625,0.410127
4,6w9lmWBgqzPQHNQo5KaORN,66EwBbt2kPgugo8Wz0SKAw,20qISvAhX20dpIbOOzGK3q,201560,True,https://api.spotify.com/v1/tracks/6w9lmWBgqzPQ...,Bonjour,43,track,0.647,...,10,"[mature, sure, spoiled, hit, south, france, to...",0.443755,0.0,0.000000,0.442664,0.000000,0.000000,0.113582,0.000000


In [77]:
df_lyrics_topics.head(50)

,track_id,album_id,artist_id,duration_ms,flg_explicit,url,track_name,track_popularity,track_type,track_danceability,...,track_release_decade,lyrics_clean_split_lemmed,Musicality/Flow/Lyricism,Flex/Money,Romance/Feelings,Struggle/Grind/Hustle,Trap/Drug,Violence/Gangster,Party/Sexuality,Feats/Artists/Shoutouts
0,7u3GcvpGFbodFKigTg7s2S,0cLzuJG2UDa0axMQkF7JR6,20qISvAhX20dpIbOOzGK3q,312893,True,https://api.spotify.com/v1/tracks/7u3GcvpGFbod...,My Country (feat. Millennium Thug),33,track,0.881,...,00,"[country, american, born, american, raised, am...",0.000000,0.000000,0.000285,0.387158,0.000000,0.042575,0.000000,0.569982
1,5ZagzwI4jeOHcy0BPQVOvy,7sHrPadM39466NY7DW3s4r,20qISvAhX20dpIbOOzGK3q,243240,True,https://api.spotify.com/v1/tracks/5ZagzwI4jeOH...,Blunt Ashes,32,track,0.777,...,00,"[blunt, ash, wonder, hughes, haley, got, told,...",0.000000,0.000000,0.125909,0.269506,0.000000,0.000000,0.042631,0.561954
2,1UgmACeNYIaoDcRFBaffs8,0jghcWTsQzux5T9sAfZO13,20qISvAhX20dpIbOOzGK3q,206466,True,https://api.spotify.com/v1/tracks/1UgmACeNYIao...,Me & You (Dedicated To Destiny),23,track,0.690,...,00,"[dedicated, destiny, like, dedicate, one, star...",0.000000,0.000000,0.000000,0.000000,0.284339,0.000000,0.000000,0.715661
3,3JiGnXAkY3d0M6Hsi6LTQ0,586sDXSThASjmWwSYnHbqP,0NbfKEOTQCcwd6o7wSDOHI,30386,False,https://api.spotify.com/v1/tracks/3JiGnXAkY3d0...,Don't Talk,13,track,0.541,...,00,"[want, think, phone, think, phone, talking, lo...",0.057499,0.000000,0.000000,0.000000,0.000000,0.318749,0.213625,0.410127
4,6w9lmWBgqzPQHNQo5KaORN,66EwBbt2kPgugo8Wz0SKAw,20qISvAhX20dpIbOOzGK3q,201560,True,https://api.spotify.com/v1/tracks/6w9lmWBgqzPQ...,Bonjour,43,track,0.647,...,10,"[mature, sure, spoiled, hit, south, france, to...",0.443755,0.000000,0.000000,0.442664,0.000000,0.000000,0.113582,0.000000
5,1rSS6Y3LWd6c7m2DA7c9Um,4UhEjfIRx4tE1XRY21vwNa,20qISvAhX20dpIbOOzGK3q,240600,True,https://api.spotify.com/v1/tracks/1rSS6Y3LWd6c...,Ghetto Prisoners,35,track,0.586,...,90,"[ghetto, regulate, nigga, nigga, live, live, g...",0.000000,0.000000,0.000000,0.796143,0.052788,0.000000,0.151069,0.000000
6,7eBnc7EiYkPUM6UAN1jzvl,4tRNl7PwUzXrxTP3YZjWgS,20qISvAhX20dpIbOOzGK3q,290474,True,https://api.spotify.com/v1/tracks/7eBnc7EiYkPU...,Royalty (ft. RaVaughn),27,track,0.676,...,10,"[royalty, whatever, young, king, wind, dead, w...",0.133884,0.000000,0.127100,0.103003,0.300768,0.335245,0.000000,0.000000
7,112VBGcsRD3hi8sFJFUiw6,7sHrPadM39466NY7DW3s4r,20qISvAhX20dpIbOOzGK3q,274266,True,https://api.spotify.com/v1/tracks/112VBGcsRD3h...,Can't Forget About You,36,track,0.642,...,00,"[forget, come, day, life, kick, back, straw, h...",0.000000,0.000000,0.000000,0.507882,0.000000,0.000000,0.300449,0.191669
8,7CxHFVZRRdP8F84nHlTbi1,3biN7VzJrWSp0qgIhaGzpn,20qISvAhX20dpIbOOzGK3q,171320,True,https://api.spotify.com/v1/tracks/7CxHFVZRRdP8...,Untitled,22,track,0.540,...,00,"[untitled, power, stop, louis, trying, touch, ...",0.000000,0.000000,0.000000,0.730845,0.000000,0.000000,0.211908,0.057247
9,4b7wW3fA6CM6iX2y3ZO2HF,6CM5qhYBvpgYNek5kYwuOJ,20qISvAhX20dpIbOOzGK3q,160161,True,https://api.spotify.com/v1/tracks/4b7wW3fA6CM6...,40 Side,55,track,0.646,...,20,"[side, hit, boy, talk, project, side, nigga, s...",0.000000,0.000000,0.509352,0.000000,0.000000,0.247658,0.000000,0.242990


In [87]:
pd.DataFrame(df_lyrics_topics.iloc[16,[6,22,36,38]]).T

,track_name,artist_name,Struggle/Grind/Hustle,Violence/Gangster
16,Life's a Bitch (feat. AZ) - Arsenal Mix,Nas,0.71268,0.215841


In [28]:
#Export songs to pickle 
df_lyrics_topics.to_pickle("df_lyrics_topics.pickle")

# Topics for Nas Songs Only

In [29]:
df_nas = df_lyrics[df_lyrics.artist_name == "Nas"]

In [30]:
text_nas = df_nas.lyrics_clean_split_lemmed.tolist()

In [42]:
#
id2word_nas, corpus_nas = get_dict_and_bow(text_nas, [0, 0.95])

In [52]:
ntopics = 6
lda_model_nas = LdaModel(corpus = corpus_nas,
                    id2word = id2word_nas,
                    num_topics = ntopics, 
                    random_state = 123,
                    passes=20,
                    alpha = 0.01,
                    eta = 0.1,
                    per_word_topics=True)

In [53]:
#Get coherence score
coherence_model_nas = get_coherence(lda_model_nas, text = text_nas, id_dict=id2word_nas)

0.2433793977272121


In [54]:
# Visualize the topics
vis_nas = pyLDAvis.gensim_models.prepare(lda_model_nas, corpus_nas, id2word_nas)
vis_nas

C:\Users\louis\anaconda3\envs\INSY669\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.002192  0.027357       1        1  24.888176
1      0.005288  0.039471       2        1  23.097908
4      0.025040  0.040925       3        1  20.300547
3      0.037198  0.044716       4        1  17.162588
0      0.067989 -0.116448       5        1   7.913011
5     -0.137706 -0.036022       6        1   6.637769, topic_info=       Term        Freq       Total Category  logprob  loglift
286   money  184.000000  184.000000  Default  30.0000  30.0000
3571  smash   64.000000   64.000000  Default  29.0000  29.0000
515    hold   88.000000   88.000000  Default  28.0000  28.0000
112    like  620.000000  620.000000  Default  27.0000  27.0000
210   world  209.000000  209.000000  Default  26.0000  26.0000
...     ...         ...         ...      ...      ...      ...
371     one   12.559158  321.393550   Topic6  -5.5420  -0.5298
212    yeah   12.369298  331.127742   Topic6  -5.5572  -0.5749
655   queen   10.887319   82.228325   Topic6  -5.6849   0.6905
99     hood   11.161706  150.481228   Topic6  -5.6600   0.1110
133   never   11.120486  253.465112   Topic6  -5.6637  -0.4140

[493 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
6055      1  0.883782      abel
5859      6  0.726837     abyss
1495      1  0.946074  accident
6011      6  0.906128    aching
5655      6  0.903252      adam
...     ...       ...       ...
3814      5  0.885075    zombie
2117      1  0.593647      zone
2117      2  0.174602      zone
2117      3  0.139682      zone
2117      6  0.104761      zone

[1020 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 2, 5, 4, 1, 6])

In [72]:
#Name the topics
topic_names_nas = {
    0:"Violence/Gangster",
    1:"Self-Evolution/Struggle",
    2:"Politics/Revolution", 
    3:"Misc",
    4:"Money",
    5:"Religion/Spirituality"
}

In [70]:
#Join topics back to dataframe
df_nas_topics = df_nas.copy()
df_nas_topics = get_topics_in_df(df = df_nas, corpus = corpus_nas, model = lda_model_nas, num_topics = 6)

In [71]:
df_nas_topics.head()

,track_id,album_id,artist_id,duration_ms,flg_explicit,url,track_name,track_popularity,track_type,track_danceability,...,lyrics_clean_stemmed,lyrics_wordlen,track_release_decade,lyrics_clean_split_lemmed,0,1,2,3,4,5
0,7u3GcvpGFbodFKigTg7s2S,0cLzuJG2UDa0axMQkF7JR6,20qISvAhX20dpIbOOzGK3q,312893,True,https://api.spotify.com/v1/tracks/7u3GcvpGFbod...,My Country (feat. Millennium Thug),33,track,0.881,...,"[countri, american, born, american, rais, amer...",405,00,"[country, american, born, american, raised, am...",0.000025,0.000025,0.000025,0.000025,0.999877,0.000025
1,5ZagzwI4jeOHcy0BPQVOvy,7sHrPadM39466NY7DW3s4r,20qISvAhX20dpIbOOzGK3q,243240,True,https://api.spotify.com/v1/tracks/5ZagzwI4jeOH...,Blunt Ashes,32,track,0.777,...,"[blunt, ash, wonder, hugh, haley, got, told, g...",275,00,"[blunt, ash, wonder, hughes, haley, got, told,...",0.000036,0.000036,0.000036,0.999818,0.000036,0.000036
2,1UgmACeNYIaoDcRFBaffs8,0jghcWTsQzux5T9sAfZO13,20qISvAhX20dpIbOOzGK3q,206466,True,https://api.spotify.com/v1/tracks/1UgmACeNYIao...,Me & You (Dedicated To Destiny),23,track,0.690,...,"[dedic, destini, like, dedic, one, star, someo...",146,00,"[dedicated, destiny, like, dedicate, one, star...",0.000068,0.262133,0.000068,0.737593,0.000068,0.000068
4,6w9lmWBgqzPQHNQo5KaORN,66EwBbt2kPgugo8Wz0SKAw,20qISvAhX20dpIbOOzGK3q,201560,True,https://api.spotify.com/v1/tracks/6w9lmWBgqzPQ...,Bonjour,43,track,0.647,...,"[matur, sure, spoil, hit, south, franc, tour, ...",133,10,"[mature, sure, spoiled, hit, south, france, to...",0.000075,0.999624,0.000075,0.000075,0.000075,0.000075
5,1rSS6Y3LWd6c7m2DA7c9Um,4UhEjfIRx4tE1XRY21vwNa,20qISvAhX20dpIbOOzGK3q,240600,True,https://api.spotify.com/v1/tracks/1rSS6Y3LWd6c...,Ghetto Prisoners,35,track,0.586,...,"[ghetto, regul, nigga, nigga, live, live, get,...",305,90,"[ghetto, regulate, nigga, nigga, live, live, g...",0.000033,0.999836,0.000033,0.000033,0.000033,0.000033


In [73]:
df_nas_topics.rename(columns = topic_names_nas, inplace = True)

In [74]:
df_nas_topics.head()

,track_id,album_id,artist_id,duration_ms,flg_explicit,url,track_name,track_popularity,track_type,track_danceability,...,lyrics_clean_stemmed,lyrics_wordlen,track_release_decade,lyrics_clean_split_lemmed,Violence/Gangster,Self-Evolution/Struggle,Politics/Revolution,Misc,Money,Religion/Spirituality
0,7u3GcvpGFbodFKigTg7s2S,0cLzuJG2UDa0axMQkF7JR6,20qISvAhX20dpIbOOzGK3q,312893,True,https://api.spotify.com/v1/tracks/7u3GcvpGFbod...,My Country (feat. Millennium Thug),33,track,0.881,...,"[countri, american, born, american, rais, amer...",405,00,"[country, american, born, american, raised, am...",0.000025,0.000025,0.000025,0.000025,0.999877,0.000025
1,5ZagzwI4jeOHcy0BPQVOvy,7sHrPadM39466NY7DW3s4r,20qISvAhX20dpIbOOzGK3q,243240,True,https://api.spotify.com/v1/tracks/5ZagzwI4jeOH...,Blunt Ashes,32,track,0.777,...,"[blunt, ash, wonder, hugh, haley, got, told, g...",275,00,"[blunt, ash, wonder, hughes, haley, got, told,...",0.000036,0.000036,0.000036,0.999818,0.000036,0.000036
2,1UgmACeNYIaoDcRFBaffs8,0jghcWTsQzux5T9sAfZO13,20qISvAhX20dpIbOOzGK3q,206466,True,https://api.spotify.com/v1/tracks/1UgmACeNYIao...,Me & You (Dedicated To Destiny),23,track,0.690,...,"[dedic, destini, like, dedic, one, star, someo...",146,00,"[dedicated, destiny, like, dedicate, one, star...",0.000068,0.262133,0.000068,0.737593,0.000068,0.000068
4,6w9lmWBgqzPQHNQo5KaORN,66EwBbt2kPgugo8Wz0SKAw,20qISvAhX20dpIbOOzGK3q,201560,True,https://api.spotify.com/v1/tracks/6w9lmWBgqzPQ...,Bonjour,43,track,0.647,...,"[matur, sure, spoil, hit, south, franc, tour, ...",133,10,"[mature, sure, spoiled, hit, south, france, to...",0.000075,0.999624,0.000075,0.000075,0.000075,0.000075
5,1rSS6Y3LWd6c7m2DA7c9Um,4UhEjfIRx4tE1XRY21vwNa,20qISvAhX20dpIbOOzGK3q,240600,True,https://api.spotify.com/v1/tracks/1rSS6Y3LWd6c...,Ghetto Prisoners,35,track,0.586,...,"[ghetto, regul, nigga, nigga, live, live, get,...",305,90,"[ghetto, regulate, nigga, nigga, live, live, g...",0.000033,0.999836,0.000033,0.000033,0.000033,0.000033


In [75]:
#Export songs to pickle 
df_nas_topics.to_pickle("df_nas_topics.pickle")